<a href="https://colab.research.google.com/github/deveshmurugan/AudioFeatureExtraction/blob/master/LTR_MSLTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import lightgbm as lgb
import pandas as pd

In [2]:
!wget https://storage.googleapis.com/personalization-takehome/MSLR-WEB10K.zip
!unzip MSLR-WEB10K.zip

--2020-12-10 17:17:23--  https://storage.googleapis.com/personalization-takehome/MSLR-WEB10K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 172.253.123.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1234144912 (1.1G) [application/zip]
Saving to: ‘MSLR-WEB10K.zip’

MSLR-WEB10K.zip     100%[===================>]   1.15G   104MB/s    in 12s     

2020-12-10 17:17:36 (95.6 MB/s) - ‘MSLR-WEB10K.zip’ saved [1234144912/1234144912]

Archive:  MSLR-WEB10K.zip
   creating: Fold1/
  inflating: Fold1/test.txt          
  inflating: Fold1/train.txt         
  inflating: Fold1/vali.txt          
   creating: Fold2/
  inflating: Fold2/test.txt          
  inflating: Fold2/train.txt         
  inflating: Fold2/vali.txt          
   creating: Fold3/
  inflating: Fold3/test.txt          
  inflating: Fold3/train.txt         
  inflating: Fo

In [3]:
def convert(input_filename, out_data_filename, out_query_filename, out_query_filename2):
	input = open(input_filename,"r")
	output_feature = open(out_data_filename,"w")
	output_query = open(out_query_filename,"w")
	output_query2 = open(out_query_filename2,"w")
	cur_cnt = 0
	cur_doc_cnt = 0
	last_qid = -1
	while True:
		line = input.readline()
		if not line:
			break
		tokens = line.split(' ')
		tokens[-1] = tokens[-1].strip()
		label = tokens[0]
		qid = int(tokens[1].split(':')[1])
		if qid != last_qid:
			if cur_doc_cnt > 0:
				output_query.write(str(cur_doc_cnt) + '\n')
				output_query2.write(str(cur_doc_cnt) + '\n')
				cur_cnt += 1
			cur_doc_cnt = 0
			last_qid = qid
		cur_doc_cnt += 1
		output_feature.write(label+' ')
		output_feature.write(' '.join(tokens[2:]) + '\n')
	output_query.write(str(cur_doc_cnt) + '\n')
	output_query2.write(str(cur_doc_cnt) + '\n')
	
	input.close()
	output_query.close()
	output_feature.close()
	output_query2.close()

In [4]:
def preprocess(): 
  train_data = lgb.Dataset(infile_train)
  valid_data = lgb.Dataset(infile_valid)

  train_group_size = [l.strip("\n") for l in open(infile_train + ".query")]
  valid_group_size = [l.strip("\n") for l in open(infile_valid + ".query")]
  train_data.set_group(train_group_size)
  valid_data.set_group(valid_group_size)

  a = pd.read_csv(path + "test.txt", sep = ' ',names=["label","qid"]+[str(c) for c in range(137)])
  b = lambda c : c.split(":")[1]
  temp = ["qid"]+list(str(c) for c in range(136))
  for x in temp:
    a[x] = a[x].apply(b)
    a[x] = pd.to_numeric(a[x], downcast='float')
  a = a.drop(columns= ['136'])

  y_test = a['label']
  x_test = a
  x_test = x_test.drop(columns = ['label'])

  return train_data, valid_data, x_test, y_test

In [5]:
def model(train_data, valid_data, x_test, y_test):
  param = {
      "task": "train",
      "num_leaves": 255,
      "min_data_in_leaf": 1,
      "min_sum_hessian_in_leaf": 100,
      "objective": "lambdarank",
      "metric": "ndcg",
      "ndcg_eval_at": [1, 3, 5, 10],
      "learning_rate": .1,
      "num_threads": 2
  }

  res = {}

  bst = lgb.train(
      param, train_data, 
      valid_sets=[valid_data], valid_names=["valid"],
      num_boost_round=50, evals_result=res, verbose_eval=10)

  pred = bst.predict(x_test)

In [6]:
infile_train = "msltr.train"
infile_valid = "msltr.vali"
path = "/content/Fold1/"
#final_accuracy = []

print("Preprocessing started for Fold 1")

convert(path + "train.txt","msltr.train","msltr.train.query","msltr.train.group")
convert(path + "vali.txt","msltr.vali","msltr.vali.query","msltr.vali.group")

train_data, valid_data, x_test, y_test = preprocess()

print("Preprocessing complete for Fold 1")

print("Model being trained for Fold 1")

model(train_data, valid_data, x_test, y_test)
print("Model Built for Fold 1!")

Preprocessing started for Fold 1
Preprocessing complete for Fold 1
Model being trained for Fold 1
[10]	valid's ndcg@1: 0.455681	valid's ndcg@3: 0.437979	valid's ndcg@5: 0.444602	valid's ndcg@10: 0.464371
[20]	valid's ndcg@1: 0.465429	valid's ndcg@3: 0.451373	valid's ndcg@5: 0.458515	valid's ndcg@10: 0.477171
[30]	valid's ndcg@1: 0.472543	valid's ndcg@3: 0.457106	valid's ndcg@5: 0.463132	valid's ndcg@10: 0.484247
[40]	valid's ndcg@1: 0.476924	valid's ndcg@3: 0.46129	valid's ndcg@5: 0.469743	valid's ndcg@10: 0.489061
[50]	valid's ndcg@1: 0.478405	valid's ndcg@3: 0.464368	valid's ndcg@5: 0.47404	valid's ndcg@10: 0.493132
Model Built for Fold 1!


This cell builds model for the remaining 4 folds.

In [7]:
infile_train = "msltr.train"
infile_valid = "msltr.vali"

for i in range(1, 6):
  path = "/content/Fold"
  path = path + str(i) + "/"
  
  print("Preprocessing started for Fold " + str(i))

  convert(path + "train.txt","msltr.train","msltr.train.query","msltr.train.group")
  convert(path + "vali.txt","msltr.vali","msltr.vali.query","msltr.vali.group")

  train_data, valid_data, x_test, y_test = preprocess()

  print("Preprocessing complete for Fold " + str(i))

  model(train_data, valid_data, x_test, y_test)
  
  print("Model Built for Fold " + str(i))

Preprocessing started for Fold 1
Preprocessing complete for Fold 1
[10]	valid's ndcg@1: 0.455681	valid's ndcg@3: 0.437979	valid's ndcg@5: 0.444602	valid's ndcg@10: 0.464371
[20]	valid's ndcg@1: 0.465429	valid's ndcg@3: 0.451373	valid's ndcg@5: 0.458515	valid's ndcg@10: 0.477171
[30]	valid's ndcg@1: 0.472543	valid's ndcg@3: 0.457106	valid's ndcg@5: 0.463132	valid's ndcg@10: 0.484247
[40]	valid's ndcg@1: 0.476924	valid's ndcg@3: 0.46129	valid's ndcg@5: 0.469743	valid's ndcg@10: 0.489061
[50]	valid's ndcg@1: 0.478405	valid's ndcg@3: 0.464368	valid's ndcg@5: 0.47404	valid's ndcg@10: 0.493132
Model Built for Fold 1
Preprocessing started for Fold 2
Preprocessing complete for Fold 2
[10]	valid's ndcg@1: 0.449619	valid's ndcg@3: 0.444051	valid's ndcg@5: 0.447898	valid's ndcg@10: 0.464068
[20]	valid's ndcg@1: 0.463395	valid's ndcg@3: 0.450637	valid's ndcg@5: 0.45537	valid's ndcg@10: 0.475868
[30]	valid's ndcg@1: 0.4771	valid's ndcg@3: 0.459334	valid's ndcg@5: 0.46354	valid's ndcg@10: 0.484109
[